#### Load Libraries

In [1]:
import numpy as np
import pandas as pd

#### Simulate Ribozyme Experiment

In [2]:
df = pd.read_csv('ribozyme_architecture.csv')

In [3]:
df.head(5)

,ID,EcoRI,OrangeForwardPrimer,BC1,Variant,PinkForwardPrimer,BC2,YellowReversePrimer,AatIIPadded,Spacer
0,ribozyme_0001,GAATTC,CGGACGTTATGGTTGGGGTT,TAGTCAAGCGC,CGTTCAGAAGCGCGCGACTGTACCGTCGTTAAGCACCTGACGTACT...,GGTGCAAGGTTGTTGGGGAA,CTTGTCAGCCC,ATCTATCTAACCGCCCCCCG,GACGTCGCG,CCTAAATTTGGAG
1,ribozyme_0002,GAATTC,CGGACGTTATGGTTGGGGTT,CATTATATGTC,TCTTCTATCATAGTTAGGCTACGGATCCTCCTCTAGGACACACCCG...,GGTGCAAGGTTGTTGGGGAA,CTTTGCTGCCG,ATCTATCTAACCGCCCCCCG,GACGTCGCG,CAG
2,ribozyme_0003,GAATTC,CGGACGTTATGGTTGGGGTT,TCGGCGCGTGA,GCTGATTCATTCCAACACTCTTGAGGCCGTTTCTATAAACGGTGAC...,GGTGCAAGGTTGTTGGGGAA,TGGGTAGTAAT,ATCTATCTAACCGCCCCCCG,GACGTCGCG,CCTAAATTTGGAG
3,ribozyme_0004,GAATTC,CGGACGTTATGGTTGGGGTT,GGAGATGATAG,CATATCTCTGTTCCTCTGCTGTAATCATGAGTCGGATCACTGGTTG...,GGTGCAAGGTTGTTGGGGAA,GCTAGGGCTTT,ATCTATCTAACCGCCCCCCG,GACGTCGCG,GCGCTGAGC
4,ribozyme_0005,GAATTC,CGGACGTTATGGTTGGGGTT,AGTGCATGTCC,GCATCCGTTGGTAACTTTCAATTGTTCAGGTCTCTGGCTTATCACG...,GGTGCAAGGTTGTTGGGGAA,CTCCGCGGTGG,ATCTATCTAACCGCCCCCCG,GACGTCGCG,ACTCTGTAAATC


In [4]:
len(df)

6232

We will assume our core molecule spans from `OrangeForwardPrimer` through to `YellowReversePrimer`. If the ribozymes are cleaved, we will assume it is cleaved after the first 10 bases into the sequences specified in the `Variant` column. We will also assume that our ribozyme activities will be linearly correlated with their rank -- the higher the rank, the more the activity in this simulation.

In [5]:
ctab = str.maketrans('ATGC', 'TACG')

In [6]:
def revcomp(seq):
    '''Return reverse complement of seq.'''
    return seq.translate(ctab)[::-1]

In [7]:
def simulate_paired_end_reads(seq, read_length):
    '''Simulate paired-end reads from seq.'''
    r1 = seq[:read_length]
    r2 = seq[-read_length:]
    return r1,revcomp(r2)

In [8]:
def attach_qualities(read, rng, q):
    '''Attach qualities to reads'''
    return ''.join(rng.choice(q, size=len(read)))

In [9]:
df.head()

,ID,EcoRI,OrangeForwardPrimer,BC1,Variant,PinkForwardPrimer,BC2,YellowReversePrimer,AatIIPadded,Spacer
0,ribozyme_0001,GAATTC,CGGACGTTATGGTTGGGGTT,TAGTCAAGCGC,CGTTCAGAAGCGCGCGACTGTACCGTCGTTAAGCACCTGACGTACT...,GGTGCAAGGTTGTTGGGGAA,CTTGTCAGCCC,ATCTATCTAACCGCCCCCCG,GACGTCGCG,CCTAAATTTGGAG
1,ribozyme_0002,GAATTC,CGGACGTTATGGTTGGGGTT,CATTATATGTC,TCTTCTATCATAGTTAGGCTACGGATCCTCCTCTAGGACACACCCG...,GGTGCAAGGTTGTTGGGGAA,CTTTGCTGCCG,ATCTATCTAACCGCCCCCCG,GACGTCGCG,CAG
2,ribozyme_0003,GAATTC,CGGACGTTATGGTTGGGGTT,TCGGCGCGTGA,GCTGATTCATTCCAACACTCTTGAGGCCGTTTCTATAAACGGTGAC...,GGTGCAAGGTTGTTGGGGAA,TGGGTAGTAAT,ATCTATCTAACCGCCCCCCG,GACGTCGCG,CCTAAATTTGGAG
3,ribozyme_0004,GAATTC,CGGACGTTATGGTTGGGGTT,GGAGATGATAG,CATATCTCTGTTCCTCTGCTGTAATCATGAGTCGGATCACTGGTTG...,GGTGCAAGGTTGTTGGGGAA,GCTAGGGCTTT,ATCTATCTAACCGCCCCCCG,GACGTCGCG,GCGCTGAGC
4,ribozyme_0005,GAATTC,CGGACGTTATGGTTGGGGTT,AGTGCATGTCC,GCATCCGTTGGTAACTTTCAATTGTTCAGGTCTCTGGCTTATCACG...,GGTGCAAGGTTGTTGGGGAA,CTCCGCGGTGG,ATCTATCTAACCGCCCCCCG,GACGTCGCG,ACTCTGTAAATC


In [10]:
def simulate_experiment(df, num_reads, read_length,fastq_prefix, seed=42):
    '''Naive ribozyme experiment simulator.'''

    # RNG Setup
    rng = np.random.default_rng(seed=seed)
    count = 0

    # Quality Setup
    q = np.array([
        '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-',
        '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':',
        ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G',
        'H', 'I'], dtype='<U1')

    # File IO Setup
    r1_file = open(f'{fastq_prefix}_R1.fq', 'w')
    r2_file = open(f'{fastq_prefix}_R2.fq', 'w')
    clx = 10

    # Prepare Products
    products = []
    for ix,row in df.iterrows():
        idx = row['ID']
        activity = ix+1
        full_seq = row['OrangeForwardPrimer'] + row['BC1'] + row['Variant'] + row['PinkForwardPrimer'] + row['BC2'] + row['YellowReversePrimer']
        cleaved_seq = (row['Variant'] + row['PinkForwardPrimer'] + row['BC2'] + row['YellowReversePrimer'])[clx:]
        assert full_seq.index(cleaved_seq) == len(row['OrangeForwardPrimer']) + len(row['BC1']) + clx
        products.append((idx, int(activity), full_seq, cleaved_seq))
    weights = np.arange(len(products)) + 1
    index = np.arange(len(products))

    # Simulation Loop
    reach  = 1
    target = 10**3
    while count <= num_reads:

        # Show Update
        if reach == target:
            reach = 0
            print(f'Simulating Read {count+1}', end='\r')
        reach += 1

        # Select Ribozyme
        (idx,
         activity,
         full_seq,
         cleaved_seq) = products[rng.choice(index, p=weights/weights.sum())]

        # Simulate Cleavage
        roll = int(rng.integers(low=1, high=len(df)+1))

        # Build Reads
        if roll < activity:
            r1,r2 = simulate_paired_end_reads(seq=cleaved_seq, read_length=read_length)
        else:
            r1,r2 = simulate_paired_end_reads(seq=full_seq, read_length=read_length)

        # Attach Qualities
        # q1 = attach_qualities(read=r1, rng=rng, q=q)
        # q2 = attach_qualities(read=r2, rng=rng, q=q)
        q1 = "I"*len(r1)
        q2 = "I"*len(r2)

        # Write Reads
        r1_file.write(f'@SimRead_{idx}_R1\n{r1}\n+\n{q1}\n')
        r2_file.write(f'@SimRead_{idx}_R2\n{r2}\n+\n{q2}\n')

        # Update Counter
        count += 1

    # Close FD
    r1_file.close()
    r2_file.close()

Simulating 100K reads.

In [11]:
simulate_experiment(
    df=df,
    num_reads=10**6,
    read_length=150,
    fastq_prefix='ribozyme_1M',
    seed=42)

Compress the files.

In [12]:
! pigz -f -11 -p48  *.fq

And we're done!

---